In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import sys
print(sys.version)

3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]


In [3]:
#datapath = '/kaggle/input/sales-dataset/'
datapath = '../data/'
filename = 'sales_data.csv'

In [4]:
#!pip install git+https://github.com/AutoViML/featurewiz_polars.git

In [5]:
import polars as pl
df = pl.read_csv(datapath+filename, null_values=['NULL','NA'], try_parse_dates=True, infer_schema_length=10000, ignore_errors=True)
print(df.shape)
df.head()

(1000, 14)


Product_ID,Sale_Date,Sales_Rep,Region,Sales_Amount,Quantity_Sold,Product_Category,Unit_Cost,Unit_Price,Customer_Type,Discount,Payment_Method,Sales_Channel,Region_and_Sales_Rep
i64,date,str,str,f64,i64,str,f64,f64,str,f64,str,str,str
1052,2023-02-03,"""Bob""","""North""",5053.97,18,"""Furniture""",152.75,267.22,"""Returning""",0.09,"""Cash""","""Online""","""North-Bob"""
1093,2023-04-21,"""Bob""","""West""",4384.02,17,"""Furniture""",3816.39,4209.44,"""Returning""",0.11,"""Cash""","""Retail""","""West-Bob"""
1015,2023-09-21,"""David""","""South""",4631.23,30,"""Food""",261.56,371.4,"""Returning""",0.2,"""Bank Transfer""","""Retail""","""South-David"""
1072,2023-08-24,"""Bob""","""South""",2167.94,39,"""Clothing""",4330.03,4467.75,"""New""",0.02,"""Credit Card""","""Retail""","""South-Bob"""
1061,2023-03-24,"""Charlie""","""East""",3750.2,13,"""Electronics""",637.37,692.71,"""New""",0.08,"""Credit Card""","""Online""","""East-Charlie"""


In [6]:
target = 'Sales_Amount'
predictors = [x for x in df.columns if x not in [target, 'Quantity_Sold']]
len(predictors)

12

In [7]:
from sklearn.model_selection import train_test_split
X = df[predictors] 
y = df[target] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
from featurewiz_polars import Featurewiz_MRMR, Featurewiz_MRMR_Model

Imported featurewiz_polars 0.1.2. Use the following syntax:
 >> from featurewiz_polars import Featurewiz_MRMR, Featurewiz_MRMR_Model
 >> wiz = Featurewiz_MRMR(model_type='Classification')
 >> X_transformed, y_transformed = wiz.fit_transform(X_train, y_train)
 >> X_test_transformed = wiz.transform(X_test)
 >> print(wiz.selected_features)
    


In [9]:
X_train.null_count()

Product_ID,Sale_Date,Sales_Rep,Region,Product_Category,Unit_Cost,Unit_Price,Customer_Type,Discount,Payment_Method,Sales_Channel,Region_and_Sales_Rep
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
mrmr = Featurewiz_MRMR_Model(model_type="Regression", 
                corr_threshold=0.7, encoding_type='onehot', classic=True, 
                verbose=0)

In [11]:
X_transformed, y_transformed = mrmr.fit_transform(X_train, y_train)


Featurewiz Polars started. Model type: Regression
SULOV selected Features (48): ['Customer_Type_New', 'Customer_Type_Returning', 'Discount', 'Payment_Method_Bank Transfer', 'Payment_Method_Cash', 'Payment_Method_Credit Card', 'Product_Category_Clothing', 'Product_Category_Electronics', 'Product_Category_Food', 'Product_Category_Furniture', 'Product_ID', 'Region_East', 'Region_North', 'Region_South', 'Region_West', 'Region_and_Sales_Rep_East-Alice', 'Region_and_Sales_Rep_East-Bob', 'Region_and_Sales_Rep_East-Charlie', 'Region_and_Sales_Rep_East-David', 'Region_and_Sales_Rep_East-Eve', 'Region_and_Sales_Rep_North-Alice', 'Region_and_Sales_Rep_North-Bob', 'Region_and_Sales_Rep_North-Charlie', 'Region_and_Sales_Rep_North-David', 'Region_and_Sales_Rep_North-Eve', 'Region_and_Sales_Rep_South-Alice', 'Region_and_Sales_Rep_South-Bob', 'Region_and_Sales_Rep_South-Charlie', 'Region_and_Sales_Rep_South-David', 'Region_and_Sales_Rep_South-Eve', 'Region_and_Sales_Rep_West-Alice', 'Region_and_Sales

In [19]:
X_train.head(2)

Product_ID,Sale_Date,Sales_Rep,Region,Product_Category,Unit_Cost,Unit_Price,Customer_Type,Discount,Payment_Method,Sales_Channel,Region_and_Sales_Rep
i64,date,str,str,str,f64,f64,str,f64,str,str,str
1049,2023-05-17,"""Alice""","""West""","""Furniture""",3290.89,3317.75,"""Returning""",0.17,"""Bank Transfer""","""Online""","""West-Alice"""
1001,2023-04-27,"""Eve""","""East""","""Electronics""",1330.62,1543.69,"""New""",0.27,"""Credit Card""","""Online""","""East-Eve"""


In [18]:
X_transformed.head(2)

Discount,Payment_Method_Bank Transfer,Payment_Method_Cash,Payment_Method_Credit Card,Product_Category_Clothing,Product_Category_Electronics,Product_Category_Food,Product_Category_Furniture,Product_ID,Region_East,Region_North,Region_South,Region_West,Region_and_Sales_Rep_East-Alice,Region_and_Sales_Rep_East-Bob,Region_and_Sales_Rep_East-Charlie,Region_and_Sales_Rep_East-Eve,Region_and_Sales_Rep_North-Alice,Region_and_Sales_Rep_North-Bob,Region_and_Sales_Rep_North-David,Region_and_Sales_Rep_North-Eve,Region_and_Sales_Rep_South-Bob,Region_and_Sales_Rep_South-Charlie,Region_and_Sales_Rep_South-David,Region_and_Sales_Rep_South-Eve,Region_and_Sales_Rep_West-Alice,Region_and_Sales_Rep_West-Bob,Region_and_Sales_Rep_West-Charlie,Region_and_Sales_Rep_West-David,Sales_Channel_Online,Sales_Channel_Retail,Sales_Rep_Alice,Sales_Rep_Bob,Sales_Rep_Charlie,Sales_Rep_David,Sales_Rep_Eve,Unit_Price,dt_Sale_Date_day,dt_Sale_Date_month,dt_Sale_Date_ordinal_day,dt_Sale_Date_weekday
f64,u8,u8,u8,u8,u8,u8,u8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,f64,i8,i8,i16,i8
0.17,1,0,0,0,0,0,1,1049.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,3317.75,17,5,137,3
0.27,0,0,1,0,1,0,0,1001.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1543.69,27,4,117,4


In [17]:
X_test_transformed = mrmr.transform(X_test)
X_test_transformed.head(2)

Discount,Payment_Method_Bank Transfer,Payment_Method_Cash,Payment_Method_Credit Card,Product_Category_Clothing,Product_Category_Electronics,Product_Category_Food,Product_Category_Furniture,Product_ID,Region_East,Region_North,Region_South,Region_West,Region_and_Sales_Rep_East-Alice,Region_and_Sales_Rep_East-Bob,Region_and_Sales_Rep_East-Charlie,Region_and_Sales_Rep_East-Eve,Region_and_Sales_Rep_North-Alice,Region_and_Sales_Rep_North-Bob,Region_and_Sales_Rep_North-David,Region_and_Sales_Rep_North-Eve,Region_and_Sales_Rep_South-Bob,Region_and_Sales_Rep_South-Charlie,Region_and_Sales_Rep_South-David,Region_and_Sales_Rep_South-Eve,Region_and_Sales_Rep_West-Alice,Region_and_Sales_Rep_West-Bob,Region_and_Sales_Rep_West-Charlie,Region_and_Sales_Rep_West-David,Sales_Channel_Online,Sales_Channel_Retail,Sales_Rep_Alice,Sales_Rep_Bob,Sales_Rep_Charlie,Sales_Rep_David,Sales_Rep_Eve,Unit_Price,dt_Sale_Date_day,dt_Sale_Date_month,dt_Sale_Date_ordinal_day,dt_Sale_Date_weekday
f64,u8,u8,u8,u8,u8,u8,u8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,f64,i8,i8,i16,i8
0.21,0,1,0,0,0,1,0,1100.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,3607.15,23,5,143,2
0.16,0,1,0,1,0,0,0,1031.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,3711.47,23,3,82,4


In [13]:
y_pred = mrmr.predict(X_test)

In [16]:
from featurewiz_polars import print_regression_metrics
print_regression_metrics(y_test.to_numpy().ravel(), y_pred, verbose=1)

    RMSE = 3177.442
    Norm RMSE = 105%
    MAE = 2773.085
    WAPE = 52%, Bias = 5.9%
    MAPE = 20047%
	Hint: high MAPE: try np.log(y) instead of (y).
    R-Squared = -9%


3177.4422973241894